# 🍀 はじめに 🍀
### 🌱このノートブックは、Stable Diffusion WebUI ForgeをGoogle Colabで手軽に起動するために制作しました。
### 🌱Google Driveへの自動保存や、Civitaiからのモデルダウンロードに対応しています。
####　🍀制作者：ざすこ（道草 雑草子） Xアカウント：https://x.com/zasuko_michiksa



# ⚒️ Stable Diffusion WebUI Forge の起動
##＜はじめる前の準備＞※重要※
### ❶生成に必要な各種モデルファイルのDL用URLを格納する各フォルダの記入欄に記載して下さい。
└🌐GoogleColab上の 📂stable-diffusion-webui-forge / models  以下に、生成に必要な各種ファイルをDLします。
### ❷（▶）ボタンをクリックしてForgeを起動


In [ ]:
# @markdown # 👈この（▶）ボタンをクリックしてForgeを起動（☕起動まで数分かかります）
# @markdown ### ※停止と再起動もこのボタンで行います。
# @markdown ## 🌱実行ログに「Running on public URL: https://...」または「This is the URL to access...」が表示されたらクリックして開いてください。

# ==========================================
# 🔧 依存関係の事前修正 (Restart回避のため最優先実行)
# ==========================================
import subprocess
print("🔧 Fixing dependencies for Forge (NumPy<2, JAX, etc)...")
# google.colabなどをimportする前に実行することで、再起動警告を回避する
subprocess.run('pip install "numpy<2" "jax>=0.4.20,<0.8.0" "jaxlib>=0.4.20,<0.8.0" "mediapipe<=0.10.15"', shell=True)

import os
import sys
import time
import threading
import socket
from pathlib import Path
from google.colab import drive
from IPython.display import clear_output

# ==========================================
# 📁 Google Drive マウント & 出力設定
# ==========================================
use_google_drive = True  # @param {type:"boolean"}

if use_google_drive:
    drive.mount('/content/drive')
    print("✅ Google Driveがマウントされました")
else:
    print("⏸️  Google Driveは使用しません")

# ==========================================
# 🛠️ Forgeのインストール
# ==========================================
%cd /content
if not os.path.exists("/content/stable-diffusion-webui-forge"):
    print("🚀 Forgeリポジトリをクローン中...")
    !git clone https://github.com/lllyasviel/stable-diffusion-webui-forge
    %cd /content/stable-diffusion-webui-forge
else:
    %cd /content/stable-diffusion-webui-forge
    !git pull

# ==========================================
# 📤 画像保存先の設定
# ==========================================
# @markdown ### 生成画像の保存先（Google Drive）
# @markdown 指定したフォルダが出力先としてリンクされます
drive_output_path = "/content/drive/MyDrive/Forge_output"  # @param {type:"string"}

if use_google_drive and drive_output_path:
    os.makedirs(drive_output_path, exist_ok=True)
    # 既存のoutputsフォルダを削除してシンボリックリンクを作成
    if os.path.exists("/content/stable-diffusion-webui-forge/outputs"):
        if not os.path.islink("/content/stable-diffusion-webui-forge/outputs"):
             !rm -rf /content/stable-diffusion-webui-forge/outputs

    if not os.path.exists("/content/stable-diffusion-webui-forge/outputs"):
         !ln -s "{drive_output_path}" /content/stable-diffusion-webui-forge/outputs
         print(f"✅ 出力先をGoogle Driveに設定しました: {drive_output_path}")

# ==========================================
# 🎁 必須パッケージ導入 & トンネル設定
# ==========================================
print("📦 Installing system dependencies...")
!apt-get -y install -qq aria2

# Cloudflaredのインストール（接続安定化のため）
!wget -q https://github.com/cloudflare/cloudflared/releases/latest/download/cloudflared-linux-amd64.deb
!dpkg -i cloudflared-linux-amd64.deb
!rm cloudflared-linux-amd64.deb
clear_output()

# ========================================
# 📥 モデルダウンロード関数定義
# ========================================

def download_with_aria2c(link, folder):
    filename = link.split("/")[-1].split('?')[0]
    # 高速化のため接続数などを調整
    # shell=Falseでリストで渡すことで引用符問題を回避
    command = [
        "aria2c", "--console-log-level=error", "-c", "-x", "16", "-s", "16", "-k", "1M",
        link, "-d", folder, "-o", filename
    ]
    print(f"Downloading {filename}...")
    try:
        subprocess.run(command, shell=False, check=True)
        print(f"✅ Downloaded: {filename}")
    except subprocess.CalledProcessError:
        print(f"❌ Failed to download: {link}")
    return filename

def download_civitai_model(civitai_link, civitai_token, folder):
    os.makedirs(folder, exist_ok=True)
    try:
        model_id = civitai_link.split("/models/")[1].split("?")[0]
    except IndexError:
        print(f"Skipping invalid Civitai URL: {civitai_link}")
        return

    # 以前は type=Model&format=SafeTensor を強制していましたが、
    # Embedding (.pt) や VAE など SafeTensor 以外のファイルが DL できない問題を修正
    # 指定なし (primary file) を要求するように変更
    civitai_url = f"https://civitai.com/api/download/models/{model_id}"

    if civitai_token:
        # トークンをURLパラメータとして付与
        civitai_url += f"?token={civitai_token}"

    # aria2cを使用して高速化を試みる
    print(f"Downloading from Civitai (with aria2c)...")
    command = [
        "aria2c", "--console-log-level=error", "-c", "-x", "16", "-s", "16", "-k", "1M",
        "-d", folder, civitai_url
    ]

    try:
        subprocess.run(command, shell=False, check=True)
        print("✅ Downloaded with aria2c")
    except subprocess.CalledProcessError:
        print("⚠️ aria2c download failed, trying wget...")
        # wget fallback (shell=Trueのまま引用符付きで実行)
        command = f'wget --content-disposition --show-progress "{civitai_url}" -P "{folder}"'
        os.system(command)

def process_urls(url_list_str, folder, token=None):
    urls = [line.strip() for line in url_list_str.split('\n') if line.strip() and not line.startswith('#')]
    if not urls:
        print(f"ℹ️ input empty for: {folder} (スキップします)")
        return

    os.makedirs(folder, exist_ok=True)
    print(f"📂 Downloading {len(urls)} files to {folder}...")

    for url in urls:
        if "civitai.com" in url:
            download_civitai_model(url, token, folder)
        else:
            download_with_aria2c(url, folder)

# ========================================
# 📥 モデルダウンロード設定
# ========================================
# @markdown ## 🔑 Civitai API Token
# @markdown Civitaiからダウンロードする場合に必要です
civitai_token = ""  # @param {type:"string"}

# @markdown ## 📁 Checkpoints (Main Models)
# @markdown 保存先: models/Stable-diffusion
checkpoint_url_1 = "https://civitai.com/api/download/models/2514310?type=Model&format=SafeTensor&size=pruned&fp=fp16"  # @param {type:"string"}
checkpoint_url_2 = "https://civitai.com/api/download/models/1176276?type=Model&format=SafeTensor&size=pruned&fp=fp16"  # @param {type:"string"}
checkpoint_url_3 = ""  # @param {type:"string"}
checkpoint_url_4 = ""  # @param {type:"string"}

checkpoints_urls = "\n".join([x for x in [checkpoint_url_1, checkpoint_url_2, checkpoint_url_3, checkpoint_url_4] if x.strip()])

# @markdown ## 📁 LoRAs
# @markdown 保存先: models/Lora
lora_url_1 = "https://civitai.com/api/download/models/135867?type=Model&format=SafeTensor"  # @param {type:"string"}
lora_url_2 = ""  # @param {type:"string"}
lora_url_3 = ""  # @param {type:"string"}
lora_url_4 = ""  # @param {type:"string"}

loras_urls = "\n".join([x for x in [lora_url_1, lora_url_2, lora_url_3, lora_url_4] if x.strip()])

# @markdown ## 📁 VAE
# @markdown 保存先: models/VAE
vae_urls = "https://civitai.com/api/download/models/333245?type=Model&format=SafeTensor"  # @param {type:"string"}

# @markdown ## 📁 Embeddings (Textual Inversion)
# @markdown 保存先: embeddings
embedding_url_1 = "https://civitai.com/api/download/models/1833157?type=Model&format=SafeTensor"  # @param {type:"string"}
embedding_url_2 = "https://civitai.com/api/download/models/2121199?type=Model&format=Other"  # @param {type:"string"}
embedding_url_3 = ""  # @param {type:"string"}
embedding_url_4 = ""  # @param {type:"string"}

embeddings_urls = "\n".join([x for x in [embedding_url_1, embedding_url_2, embedding_url_3, embedding_url_4] if x.strip()])

# @markdown ## 📁 ControlNet Models
# @markdown 保存先: models/ControlNet
controlnet_url_1 = "https://huggingface.co/xinsir/controlnet-union-sdxl-1.0/resolve/main/diffusion_pytorch_model_promax.safetensors?download=true"  # @param {type:"string"}
controlnet_url_2 = "https://huggingface.co/lllyasviel/sd_control_collection/resolve/main/thibaud_xl_openpose.safetensors?download=true"  # @param {type:"string"}
controlnet_url_3 = "https://huggingface.co/GSennin/controlnets/resolve/main/noobaiXLControlnet_openposeModel.safetensors?download=true"  # @param {type:"string"}
controlnet_url_4 = ""  # @param {type:"string"}

controlnet_urls = "\n".join([x for x in [controlnet_url_1, controlnet_url_2, controlnet_url_3, controlnet_url_4] if x.strip()])

# ダウンロード実行
process_urls(checkpoints_urls, "/content/stable-diffusion-webui-forge/models/Stable-diffusion", civitai_token)
process_urls(loras_urls, "/content/stable-diffusion-webui-forge/models/Lora", civitai_token)
process_urls(vae_urls, "/content/stable-diffusion-webui-forge/models/VAE", civitai_token)
process_urls(embeddings_urls, "/content/stable-diffusion-webui-forge/embeddings", civitai_token)
process_urls(controlnet_urls, "/content/stable-diffusion-webui-forge/models/ControlNet", civitai_token)
print("✅ ダウンロード処理完了")

# ========================================
# 🧩 拡張機能のインストール
# ========================================
# @markdown ## 🧩 Extensions (Optional)
# @markdown 起動時に自動インストールする拡張機能を選択してください
install_prompt_all_in_one = True  # @param {type:"boolean"}
install_adetailer = True  # @param {type:"boolean"}
install_aspect_ratio_plusplus = True  # @param {type:"boolean"}

extensions_dir = "/content/stable-diffusion-webui-forge/extensions"
os.makedirs(extensions_dir, exist_ok=True)

if install_prompt_all_in_one:
    repo_path = f"{extensions_dir}/sd-webui-prompt-all-in-one"
    if not os.path.exists(repo_path):
        print("🧩 Installing sd-webui-prompt-all-in-one...")
        !git clone https://github.com/Physton/sd-webui-prompt-all-in-one.git {repo_path}
    else:
        print("🧩 sd-webui-prompt-all-in-one already installed.")

if install_adetailer:
    repo_path = f"{extensions_dir}/adetailer"
    if not os.path.exists(repo_path):
        print("🧩 Installing adetailer...")
        !git clone https://github.com/Bing-su/adetailer.git {repo_path}
    else:
        print("🧩 adetailer already installed.")

if install_aspect_ratio_plusplus:
    repo_path = f"{extensions_dir}/sd-webui-ar-plusplus"
    if not os.path.exists(repo_path):
        print("🧩 Installing sd-webui-ar-plusplus (Aspect Ratio ++)...")
        !git clone https://github.com/altoiddealer/--sd-webui-ar-plusplus.git {repo_path}
    else:
        print("🧩 sd-webui-ar-plusplus already installed.")

# ========================================
# 🚀 Forge 起動 & 接続監視
# ========================================

# Cloudflared/Gradioの起動監視スレッド
def iframe_thread(port):
    while True:
        time.sleep(0.5)
        sock = socket.socket(socket.AF_INET, socket.SOCK_STREAM)
        result = sock.connect_ex(('127.0.0.1', port))
        if result == 0:
            break
        sock.close()

    print("\n🚀 Forge started! Setting up tunnel...\n")

    # Cloudflaredトンネルの作成
    p = subprocess.Popen(["cloudflared", "tunnel", "--url", "http://127.0.0.1:{}".format(port)], stdout=subprocess.PIPE, stderr=subprocess.PIPE)
    for line in p.stderr:
        l = line.decode()
        if "trycloudflare.com " in l:
            cf_url = l[l.find("http"):].strip()
            print("\n" + "="*60)
            print(f"🔥 Cloudflare URL: {cf_url}")
            print("="*60 + "\n")

# ポート7860（Forgeデフォルト）を監視開始
threading.Thread(target=iframe_thread, daemon=True, args=(7860,)).start()

print("🚀 Forgeを起動します...初期セットアップには数分かかる場合があります。")

# 起動引数
args = [
    "--share",
    "--theme dark",
    "--enable-insecure-extension-access",
    "--cuda-malloc",
    "--cuda-stream"
]

args_str = " ".join(args)

# launch.pyの実行
!python launch.py {args_str}


# 📝 メモ・リンク集
## おすすめモデルなど
- [Civitai](https://civitai.com/)
- [Hugging Face](https://huggingface.co/)
- [Stable Diffusion WebUI Forge (GitHub)](https://github.com/lllyasviel/stable-diffusion-webui-forge)
